## BONUS: Introducción a Prefect

## Paso 1: Setup Básico

In [ ]:
# === SETUP PREFECT ===

# Instalar Prefect (si no está instalado)
# !pip install prefect

import prefect
from prefect import task, flow, get_run_logger
import pandas as pd

print("Prefect instalado y configurado")
print(f"   Versión: {prefect.__version__}")

## Paso 2: Convertir funciones a tasks

In [ ]:
# === TASKS SIMPLES PARA APRENDER PREFECT ===

@task(name="Cargar Datos", retries=2, retry_delay_seconds=3)
def cargar_datos(url: str, tipo: str) -> pd.DataFrame:
    """Task simple para cargar cualquier tipo de datos"""
    logger = get_run_logger()
    logger.info(f"Cargando {tipo} desde: {url}")

    # Cargar según el tipo
    if tipo == "trips":
        data = pd.read_parquet(url)  # función para Parquet
    elif tipo == "zones":
        data = pd.read_csv(url)      # función para CSV
    else:  # calendar
        data = pd.read_json(url)     # función para JSON
        data['date'] = pd.to_datetime(data['date']).dt.date  # convertir strings a fechas

    logger.info(f"{tipo} cargado: {data.shape[0]} filas")
    return data


@task(name="Hacer Join Simple")
def hacer_join_simple(trips: pd.DataFrame, zones: pd.DataFrame) -> pd.DataFrame:
    """Task para hacer join básico de trips + zones"""
    logger = get_run_logger()
    logger.info("Haciendo join simple...")

    # Normalizar columnas
    trips.columns = trips.columns.str.lower()   # convertir a minúsculas
    zones.columns = zones.columns.str.lower()   # misma transformación

    # Join básico
    resultado = trips.merge(zones,              # método para unir DataFrames
                            left_on='pulocationid',   # columna de pickup location en trips
                            right_on='locationid',    # columna de location en zones
                            how='left')               # tipo de join que mantiene todos los trips

    logger.info(f"Join completado: {len(resultado)} registros")
    return resultado


@task(name="Análisis Rápido")
def analisis_rapido(data: pd.DataFrame) -> dict:
    """Task para análisis básico"""
    logger = get_run_logger()
    logger.info("Haciendo análisis básico...")

    # Stats simples
    stats = {
        'total_registros': len(data),  # método para contar valores
        'boroughs': data['borough'].value_counts().head(3).to_dict(),  # contar valores únicos
        'distancia_promedio': round(data['trip_distance'].mean(), 2),  # método para promedio
        'tarifa_promedio': round(data['total_amount'].mean(), 2)       # método para promedio
    }

    logger.info(f"Análisis completado: {stats['total_registros']} registros")
    return stats

## Paso 3: Crear un flow simple

In [ ]:
# === FLOW PRINCIPAL (EL PIPELINE COMPLETO) ===

@flow(name="Pipeline Simple NYC Taxi")
def pipeline_taxi_simple():
    """Flow simple que conecta todos los tasks"""

    logger = get_run_logger()
    logger.info("Iniciando pipeline simple...")

    # URLs de datos
    trips_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
    zones_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv"

    # PASO 1: Cargar datos (con retry automático si falla)
    logger.info("Paso 1: Cargando datos...")
    trips = cargar_datos(trips_url, "trips")    # tipo de datos = trips
    zones = cargar_datos(zones_url, "zones")    # tipo de datos = zones

    # PASO 2: Hacer join
    logger.info("Paso 2: Haciendo join...")
    data_unida = hacer_join_simple(trips, zones)

    # PASO 3: Análisis básico
    logger.info("Paso 3: Analizando...")
    resultados = analisis_rapido(data_unida)

    # PASO 4: Mostrar resultados
    logger.info("Pipeline completado!")
    logger.info(f"Resultados: {resultados}")

    return resultados

## Paso 4: Ejecutar el Pipeline

In [ ]:
# === EJECUTAR EL PIPELINE ===

if __name__ == "__main__":
    print("🚀 Ejecutando pipeline simple...")

    # Ejecutar el flow
    resultado = pipeline_taxi_simple()   # nombre de la función del flow

    print("\n📊 RESULTADOS FINALES:")
    print(f" Total registros: {resultado['total_registros']}")
    print(f" Distancia promedio: {resultado['distancia_promedio']} millas")
    print(f" Tarifa promedio: ${resultado['tarifa_promedio']}")

    print("\n🏙️ Top 3 Boroughs:")
    for borough, count in resultado['boroughs'].items():  # clave del diccionario que contiene boroughs
        print(f" {borough}: {count} viajes")